# Why Backtesting Matters and How to do it Right

The related article is available [here](https://towardsdatascience.com/why-backtesting-matters-and-how-to-do-it-right-731fb9624a).

This notebook showcases how to backtest a time-series forecasting model.

To install the needed dependencies, you can follow the instructions in the README at the root of the repository.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from lightgbm import LGBMRegressor

## Data Preparation

Let's use the Air Passengers dataset for this example: monthly totals of international airline passengers, 1949 to 1960.

In [ ]:
# Load data.
data = pd.read_csv('https://raw.githubusercontent.com/unit8co/darts/master/datasets/AirPassengers.csv')
# Rename columns
data = data.rename(columns = {"Month": "time", "#Passengers": "passengers"})
# Set time to datetime
data.time = pd.to_datetime(data.time)
# Set time as index.
data = data.set_index("time")

## Quick Analysis

In [ ]:
# Let's visualize the data.
def show_data(data,title=""):
    trace = [go.Scatter(x=data.index,y=data[c],name=c) for c in data.columns]
    go.Figure(trace,layout=dict(title=title)).show()

show_data(data,"Monthly totals of international airline passengers")

We can see that the data exhibits a strong increasing trend and yearly seasonality.

## Data Engineering

Let's predict the value of the next month based on:
- the lagged values of the previous 2 years.
- the current month (as a categorical feature).

In [ ]:
def build_target_features(data, lags, horizon=1):
    # Build lagged features.
    feat = pd.concat(
        [
            data[["passengers"]].shift(lag).rename(columns={"passengers": f"lag_{lag}"})
            for lag in lags
        ],
        axis=1,
    )
    
    # Build month feature.
    feat["month"] = [f"M{m}" for m in data.index.month]
    feat["month"] = feat["month"].astype("category")

    # Build target at horizon.
    targ = data["passengers"].shift(-horizon).rename(f"horizon_{horizon}")

    # Drop missing values generated by lags/horizon.
    idx = ~(feat.isnull().any(axis=1) | targ.isnull())
    feat = feat.loc[idx]
    targ = targ.loc[idx]

    return targ, feat


# Build targets and features.
target, features = build_target_features(
    data,
    lags=[0,1,2,5,11,23],
    horizon=1,
)

## Backtesting implementation

In [ ]:
def run_backtest(
    model,
    target,
    features,
    start_window = 10,
    retrain_frequency = 6,
):
    """Simple backtesting implementation.
    
    Args:
        model: A model with fit and predict methods.
        targets: Series with the target in chronological order.
        features: Dataframe with the features in chronological order.
        start_window: The initial window to train a model.
        retrain_frequency: How often to retrain the model.
        
    Return:
        A dataframe with the validation target and prediction.
    """
    
    # Sanity check on shape
    assert features.shape[0] == target.shape[0]
    
    
    all_preds = []
    all_targets = []
    last_timestep = start_window
    while last_timestep < len(target):

        # Split train/valid
        targ_train = target.iloc[:last_timestep]
        feat_train = features.iloc[:last_timestep]
        targ_valid = target.iloc[last_timestep:last_timestep+1]    
        feat_valid = features.iloc[last_timestep:last_timestep+1]

        # Train the model
        if last_timestep==start_window or last_timestep % retrain_frequency == 0:
            model.fit(feat_train,targ_train)

        # Predict on valid set
        pred_valid = model.predict(feat_valid)

        # Save the output
        all_preds.append(pred_valid[0])
        all_targets.append(targ_valid)

        # Process next timestep
        last_timestep += 1

    # Format output
    output = pd.concat(all_targets).rename("target").to_frame()
    output["prediction"] = all_preds

    return output

## Check backtesting output

In [ ]:
# Apply run_backtest to our data.
output_backtest = run_backtest(
    LGBMRegressor(min_child_samples=1, objective="mae"),
    target,
    features,
    start_window = 36,
    retrain_frequency = 6,
)

# First, let's visualize the data.
show_data(output_backtest,"Backtested predictions at horizon 1")

# Then, let's compute the MAE.
MAE = abs(output_backtest.prediction - output_backtest.target).mean()
print(f"MAE: {MAE:.1f}")